# Quick Start

Bridgic is an innovative framework for agent programming and dynamic task orchestration. It offers developers a brand-new programming paradigm. Whether it's for rapid experimentation, complex system orchestration, or building intelligent agents with long lifecycles, Bridgic provides concise and powerful support.

## Word Learning Assistant

In this tutorial, let's build a sample word learning assistant. Input a word, output its derivational variations and make sentences using these variations. At the same time, we will understand the sample usage of Bridgic along with the word learning assistant.

### 1. Initialize

Before we start, let's prepare the running environment.

In [ ]:
# Get the environment variables.
import os
from dotenv import load_dotenv
load_dotenv()

_api_key = os.environ.get("OPENAI_API_KEY")
_model_name = os.environ.get("OPENAI_MODEL_NAME")  # Using gpt-4.1-mini in this example

# Import the necessary packages.
from bridgic.core.automa import GraphAutoma, worker
from bridgic.core.model.types import Message, Role
from bridgic.llms.openai.openai_llm import OpenAILlm

llm = OpenAILlm(
    api_key=_api_key,
    timeout=20,
)

Bridgic provides a powerful encapsulation for model usage. Here, we will simply use it first. If you want to know more details, you can refer to this tutorial: [Model Usage](model_usage.ipynb).

### 2. Complete

There are two steps to complete the word learning assistant:

1. Generate derivatives of the input word.
2. Make sentences with derivatives.

In [9]:
class WordLearningAssistant(GraphAutoma):
    @worker(is_start=True)
    async def generate_derivatives(self, word: str):
        print(f"------Generating derivatives for {word}------")
        response = await llm.achat(
            model=_model_name,
            messages=[
                Message.from_text(text="You are a word learning assistant. Generate derivatives of the input word in a list.", role=Role.SYSTEM),
                Message.from_text(text=word, role=Role.USER),
            ]
        )
        print(response.message.content)
        print(f"------End of generating derivatives------\n")
        return response.message.content

    @worker(dependencies=["generate_derivatives"], is_output=True)
    async def make_sentences(self, derivatives):
        print(f"------Making sentences with------")
        response = await llm.achat(
            model=_model_name,
            messages=[
                Message.from_text(text="You are a word learning assistant. Make sentences with the input derivatives in a list.", role=Role.SYSTEM),
                Message.from_text(text=derivatives, role=Role.USER),
            ]
        )
        print(response.message.content)
        print(f"------End of making sentences------\n")
        return response.message.content

Let's run it!

In [10]:
word_learning_assistant = WordLearningAssistant()
res = await word_learning_assistant.arun(word="happy")

------Generating derivatives for happy------
1. happiness  
2. happily  
3. happier  
4. happiest  
5. unhappiness  
6. unhappily  
7. unhappier  
8. unhappiest
------End of generating derivatives------

------Making sentences with------
1. Happiness is the key to a fulfilling life.  
2. She happily accepted the invitation to the party.  
3. I'm happier now that I've started a new job.  
4. That was the happiest day of my childhood.  
5. His unhappiness became apparent after the breakup.  
6. They lived unhappily together for many years.  
7. After the storm, the plants looked unhappier than before.  
8. She felt the unhappiest when she lost her favorite book.
------End of making sentences------



> Note: Ensure you have set up your .env file to store your OPENAI_API_KEY or set up your terminal environment variable. This key is necessary for authenticating requests to the OpenAI API.

Great! We have successfully completed the word learning assistant. It correctly completed the task as per our requirements.

<div style="text-align: center; margin: 2rem 0;">
<hr style="border: none; border-top: 2px solid #e2e8f0;">
</div>

## What have we done?

The above example is the typical usage of writing an agent application with Bridgic. Now let's understand some of its components.

### Worker

Any callable object (such as functions, methods, etc.) when used by the framework, will be converted into a worker object and serve as the **smallest execution unit** for scheduling and orchestration. 

Just as in the example of the word learning assistant, we can use decorator syntax to wrap functions and methods into a worker object.

In [ ]:
class MyAutoma(GraphAutoma):
    @worker(is_start=True)
    async def start(self, x: int):
        return x    

Or, you can also use it like this:

In [ ]:
class MyAutoma(GraphAutoma): ...
my_automa = MyAutoma()

# Add the function as a worker with worker decorator in the instance of the automa
@my_automa.worker(is_start=True)
async def start(x: int):
    return x


Another one, the interface `add_func_as_worker()` can also be used to add workers.

In [ ]:
async def start(x: int):
    return x

class MyAutoma(GraphAutoma): ...
my_automa = MyAutoma()

# Add the function as a worker
my_automa.add_func_as_worker(
    key="start",
    func=start,
    is_start=True,
)


In addition to functions being convertible to workers, classes that inherit from `Worker` and override either `run()` or `arun()` can also be used directly as workers by `add_worker()` interface.

In [ ]:
from Bridgic.core.automa.worker import Worker

class MyWorker(Worker):
    async def arun(self, x: int):
        return x

my_worker = MyWorker()

# Add the worker to the automa
class MyAutoma(GraphAutoma): ...
my_automa = MyAutoma()
my_automa.add_worker(
    key="my_worker",
    worker=my_worker,
    is_start=True,
)

# Run the worker
res = await my_automa.arun(x=1)
print(res)

> Note:
> 1. A specific worker that inherits from `Worker` must override either the `run()` or `arun()` method.
> 2. Bridgic is a framework primarily designed for asynchronous execution, if both `run()` and `arun()` of a worker are overridden, `arun()` will take precedence.

Both of these ways can correctly add the workers to `MyAutoma`. 

Whether using decorator syntax or the corresponding interface, there are usually some parameters:

1. `key`: A string. As the unique identifier of a worker in the current automa, it must be ensured that there are no duplicate names within the same automa. Use function names or class names by default.
2. `func`(in `add_func_as_worker()`) or `worker`(in `add_worker()`): The actual callable object. **The decorator syntax does not have this parameter.**
3. `is_start`: `True` or `False`. Marking the worker as the start for automa. It can be set in multiple workers.
4. `dependencies`: A list of string. Mark the preceding workers that the worker depends on.
5. `is_output`: `True` or `False`. Marking the worker as the output for automa. There can only be one on an execution branch.
6. `args_mapping_rule`: Parameter passing rule. For detailed information on behavior classes, please refer to the tutorial: [Parameter Passing](./parameter_passing.ipynb)

> Note: From the perspective of the Bridgic, a worker must be placed in an automa for scheduling before it can be executed. Of course, even after packaging it as a worker, you can directly call `worker.arun()` or `worker.run()` to run it, but this is not within the purview of Bridgic.

### Automa

Automa serves as the **orchestration engine**. Developers can entrust multiple workers to automa for management, and it is responsible for unified scheduling and operation, acting as the entry point for the entire process.

Different types of automa represent different orchestration rules. In the example of the word learning assistant above, we used the `GraphAutoma`, which represents the orchestration and scheduling according to the topological sorting among workers.

You can use it by writing a class that inherits from it and writing or adding workers.

In [3]:
# Write workers in MyAutoma
class MyAutoma(GraphAutoma):
    @worker(is_start=True)
    async def worker_0(self, a, b, x, y):
        print(f"worker_0: a={a}, b={b}, x={x}, y={y}")

    @worker(is_start=True)
    async def worker_1(self, x, y):
        print(f"worker_1: x={x}, y={y}")

After an automa has the required workers, it can call `await automa_obj.arun(*args, **kwargs)` to start the entire scheduling operation. 

> Bridgic is an asynchronous framework, `Graphautoma` must be started using arun().

At startup, the parameters of `automa_obj.arun(*args, **kwargs)` will be distributed to the worker with `is_start=True` according to positional parameters and keyword parameters. 

- positional parameters: The positional parameters will be filled into the parameter list of the worker with `is_start=True` in the order of input. An error will be raised if the parameter list of some worker is shorter than the number of positional parameters.
- keyword parameters: The keyword parameter will be filled into the corresponding parameter of the corresponding worker with `is_start=True`.
- priority: **Positional parameters have a higher priority than keyword parameters**.

For example: we pass positional arguments `1` and `2`, and keyword arguments `x=3`, `y=4`.

In [4]:
my_automa = MyAutoma()
await my_automa.arun(1, 2, x=3, y=4)

worker_0: a=1, b=2, x=3, y=4
worker_1: x=1, y=2


`1` and `2` were received in sequence by the first and second parameters of `worker_0` and `worker_1` respectively. Because positional parameters have a higher priority than keyword parameters, even if the parameter names of `worker_1` are the same as the input keyword parameters, they will still preferentially receive positional parameters.

An error will be raised if the parameter list of some worker is shorter than the number of positional parameters.

In [ ]:
my_automa = MyAutoma()
await my_automa.arun(1, 2, 3, y=4)  # error

If all parameters are input in the keyword parameters, each worker with `is_start=True` can receive the corresponding parameter value.

In [10]:
my_automa = MyAutoma()
await my_automa.arun(a=1, b=2, x=3, y=4)

worker_0: a=1, b=2, x=3, y=4
worker_1: x=3, y=4


Now, we can start building our Bridgic project!